In [1]:
from os import listdir
import os, random, copy
from PIL import Image
import numpy as np
from collections import defaultdict
import math

#define sigmoid and its derivative for activation & backprop
def sigmoid(x):
    return 1 / (1 + math.exp(-1 * x))

def derivSigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [2]:
################################################################################
# CSE 253: Programming Assignment 1
# Code snippet by Michael
# Winter 2020
################################################################################
# We've provided you with the dataset in PA1.zip
################################################################################
# To install PIL, refer to the instructions for your system:
# https://pillow.readthedocs.io/en/5.2.x/installation.html
################################################################################
# If you don't have NumPy installed, please use the instructions here:
# https://scipy.org/install.html
################################################################################


''' 
list of face expressions (contempt, neutral are excluded) are:
1. anger
2. disgust
3. fear
4. happiness
5. sadness
6. surprise
'''

def load_data(data_dir="./aligned/"):
	""" Load all PNG images stored in your data directory into a list of NumPy
	arrays.

	Args:
		data_dir: The relative directory path to the CK+ image directory.
	Returns:
		images: A dictionary with keys as emotions and a list containing images associated with each key.
		cnt: A dictionary that stores the # of images in each emotion
	"""
	images = defaultdict(list)

	# Get the list of emotional directory:
	for e in listdir(data_dir):
		# excluding any non-directory files
		if not os.path.isdir(os.path.join(data_dir, e)):
			continue
		# Get the list of image file names
		all_files = listdir(os.path.join(data_dir, e))

		for file in all_files:
			# Load only image files as PIL images and convert to NumPy arrays
			if '.png' in file:
				img = Image.open(os.path.join(data_dir, e, file))
				images[e].append(np.array(img))

	print("Emotions: {} \n".format(list(images.keys())))

	cnt = defaultdict(int)
	for e in images.keys():
		print("{}: {} # of images".format(e, len(images[e])))
		cnt[e] = len(images[e])
	return images, cnt

def balanced_sampler(dataset, cnt, emotions):
	# this ensures everyone has the same balanced subset for model training, don't change this seed value
	random.seed(20)
	print("\nBalanced Set:")
	min_cnt = min([cnt[e] for e in emotions])
	balanced_subset = defaultdict(list)
	for e in emotions:
		balanced_subset[e] = copy.deepcopy(dataset[e])
		random.shuffle(balanced_subset[e])
		balanced_subset[e] = balanced_subset[e][:min_cnt]
		print('{}: {} # of images'.format(e, len(balanced_subset[e])))
	return balanced_subset

def display_face(img):
	""" Display the input image and optionally save as a PNG.

	Args:
		img: The NumPy array or image to display

	Returns: None
	"""
	# Convert img to PIL Image object (if it's an ndarray)
	if type(img) == np.ndarray:
		print("Converting from array to PIL Image")
		img = Image.fromarray(img)

	# Display the image
	img.show()


# example on how to use it
if __name__ == '__main__':
	# The relative path to your image directory
	data_dir = "./aligned/"
	dataset, cnt = load_data(data_dir)
	# test with happiness and anger
	images = balanced_sampler(dataset, cnt, emotions=['happiness', 'anger'])
	display_index = 0
	display_face(images['anger'][display_index])

Emotions: ['fear', 'surprise', 'sadness', 'happiness', 'anger', 'disgust'] 

fear: 25 # of images
surprise: 83 # of images
sadness: 28 # of images
happiness: 69 # of images
anger: 45 # of images
disgust: 59 # of images

Balanced Set:
happiness: 45 # of images
anger: 45 # of images
Converting from array to PIL Image


In [3]:
#list happiness as a 1, anger as a 0
X = images['happiness'] + images['anger']
y = [1] * 45 + [0] * 45

#get my feature - label pairs zipped together
all_happy_angry = list(zip(X,y))

#randomize the dataset so I can fold properly

random.shuffle(all_happy_angry)

def kfold(data, k=10):
    folds = []
    size = int((1/k) * len(data))
    for i in range(0,k):
        folds.append(data[i*size : (i+1)*size])
    return folds

In [51]:
# Need np.linalg.svd somewhere
#The rows of vh are the eigenvectors of A^H A and 
#the columns of u are the eigenvectors of A A^H. 
#In both cases the corresponding (possibly non-zero) 
#eigenvalues are given by s**2.

#images are 224 x 192
#n_sq = 43008



def PCA(training_data): # Requires 3d input [example, row, column]
    td = np.array(training_data)
    print("Input dimensions:")
    print(td.shape)
    num_examples, rows, columns = td.shape

    n_sq = rows * columns
    
    Iota = np.reshape(td, (num_examples, n_sq))
    print("Iota dimensions:")
    print(Iota.shape)

    mean_face = np.mean(Iota, axis=0)
    print("Mean face dimensions:")
    print(mean_face.shape)
    
    Iota_norm = Iota - mean_face
    A = Iota_norm
    A = A.T
    print("A shape")
    print(A.shape)

    
    C = np.matmul(A.T, A)
    print("C shape: ")
    print(C.shape)
    evals, Vi = np.linalg.eigh(C)
    print("Evals then Evecs:")
    print(evals.shape)
    print(Vi.shape)
    #u, s, vh = np.linalg.svd(C)
    
    
        
    #sort eigenvectors based on eigenvalues
    z = list(zip(evals,Vi))
    z.sort(reverse=True)
    Vi = np.array([x[1] for x in z])
    
    #multiply A by V to find eigenvectors of c
    evecs = np.matmul(A, Vi)
    print("eigv shape")
    print(evecs.shape)
    
    
    return evecs, evals
    '''
    pc_norm_singular_values = np.sqrt(evals.reshape(1,-1))
    print("pc sing val shape:")
    print(pc_norm_singular_values.shape)
    pc_norm = eigv / np.linalg.norm(eigv, axis=0)
    print("pc norm shape:")
    print(pc_norm.shape)
    return pc_norm, pc_norm_singular_values
    '''
    
    

In [52]:
pca, sv = PCA([x[0] for x in all_happy_angry])

print(pca[:,0])
#img = Image.fromarray(np.reshape( np.matmul(pca[:,0],np.reshape(all_happy_angry[0][0], (1,43008))) , (224, 192)))
#img = Image.fromarray(vh)
img.show()

Input dimensions:
(90, 224, 192)
Iota dimensions:
(90, 43008)
Mean face dimensions:
(43008,)
A shape
(43008, 90)
C shape: 
(90, 90)
Evals then Evecs:
(90,)
(90, 90)
eigv shape
(43008, 90)
[-1.76991883e-12 -1.70196563e-12 -1.71031681e-12 ... -1.37758812e-12
 -1.60386822e-12 -1.54615179e-12]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 43008)